In [1]:
print("=== PROCEDURE, REFERENCE, & LIMITATIONS ===")
print("Procedure: Direct Design Method")
print("Reference: ACI 318M-14 | RCD by Darwin & Nelson")
print("Limitations: \n minimum 3x3 panel. If fewer, interior negative moments tend to be too small. \n Ratio not greater than 2. \n Span lengths not differ by more than 1/3 of the longer span. \n applicable only to braced frames \n Service live load shall not exceed two times the service dead load. \n relative stiffness of beams in two perpedicular directions shall not be lt 0.2 or gt 5.0.")

=== PROCEDURE, REFERENCE, & LIMITATIONS ===
Procedure: Direct Design Method
Reference: ACI 318M-14 | RCD by Darwin & Nelson
Limitations: 
 minimum 3x3 panel. If fewer, interior negative moments tend to be too small. 
 Ratio not greater than 2. 
 Span lengths not differ by more than 1/3 of the longer span. 
 applicable only to braced frames 
 Service live load shall not exceed two times the service dead load. 
 relative stiffness of beams in two perpedicular directions shall not be lt 0.2 or gt 5.0.


In [2]:
import math
pi = math.pi
import pandas as pd
def sqrt(x):
    return math.sqrt(x)
def rd(x):
    return round(x,2)
def roundup(x):
    return math.ceil(x/5.0) * 5

In [3]:
#conversions
def mm_to_in(x):
    return round(x/25.4,2)
def kPa_to_psf(x):
    return round(x*144/6.89655172413793,2)
def psf_to_kPa(x):
    return round(x*6.89655172413793/144,2)
def kNm_to_kipft(x):
    """
    Output:
    Converts kN-m to kip-ft
    """
    return rd(x/1.35582)
def kipft_to_kNm(x):
    """
    Output:
    Converts kip-ft to kN-m
    """
    return rd(x*1.35582)

In [4]:
#Class and methods
class Beam:
    def __init__ (self, name, dimy, dimz, length, loc):
        """
        Creates a beam object

        Parameters:
        name: beam name
        dimy (mm): bw or width of the beam
        dimz (mm): ht or height of the beam
        length (mm): length of the beam
        loc (str): if "edge" beam or "interior" beam
        
        Returns:
        str: Generates a beam object
        """
        self.name = name
        self.dimy = dimy #bw, mm
        self.dimz = dimz #ht, mm
        self.length  = length #mm
        self.loc = loc
        
    def Icb(self):
        if self.loc == "edge":
            return (self.dimy*self.dimz**3/12)*1.5
        elif self.loc == "interior":
            return (self.dimy*self.dimz**3/12)*2.0
        
    #Relative restraint provided by the torsional resistance of the eff. transv. edge beam

    def C(self,slab_thk): #Nelson, pp415
        hw = self.dimz - slab_thk
        flange = min(hw, 4*slab_thk)
        C = (1 - 0.63*self.dimy/self.dimz)*(self.dimy**3)*(self.dimz/3) + (1 - 0.63*slab_thk/flange)*(slab_thk**3)*(flange/3) #mm4
        return C
    
    def beta_t(self,slab_thk,Ics):
        """
        ratio of torsional stiffness of edge beam section to flexural stiffness of a width of slab equal to span length of the beam, c2c of supports
        = C of edge beam / flexStif of slab w/ width = lgth of edge beam
        Parameters:
        slab_thk (mm): slab thickness
        Ics (mm4): MoI of slab (having width = lgth of edge beam)

        """
        return self.C(slab_thk)/(2*Ics)
    
    def alpha_f(self, Ics):
        """
        Returns:
        returns the relative stiffness of beam and slab.
        """
        stiff = rd( self.Icb() / Ics)
        print(f"{self.name}, alpha_f = {stiff}")
        return stiff

class Slab:
    def __init__ (self, name, L, W, thk):
        """
        Creates a slab object

        Parameters:
        name: slab name
        L (mm): longer clear span of the slab
        W (mm): shorter clear span of the slab
        thk (mm): slab thickness
        
        Returns:
        str: Generates a slab object
        """
        self.name = name
        self.L = L #The larger dimension
        self.W = W 
        self.thk = thk

    def Ics_W_half(self):
        return self.W*0.5*self.thk**3/12
    
    def Ics_L_half(self):
        return self.L*0.5*self.thk**3/12
    
    def Ics_W_whole(self):
        return self.W*self.thk**3/12
    
    def Ics_L_whole(self):
        return self.L*self.thk**3/12

In [5]:
#Slab and Beam Material Properties
fc = 27 #MPa
Ec = rd(4700*sqrt(fc)/1000) #GPa, for normal weight concrete
fy = 230 #MPa
Es = 200000     #Steel Elasticity, MPa
print("=== MATERIAL PROPERTIES ===")
print(f"fc = {fc}MPa | fy = {fy}MPa | Ec = {Ec}GPa")

#Beam Material and Geometric Properties
B_nort = Beam("B_north",350,650,5700,"interior")
B_sout = Beam("B_south",350,650,5700,"interior")
B_east = Beam("B_east",350,650,2700,"interior")
B_west = Beam("B_west",350,650,2700,"edge")

#Slab Material and Geometric Properties
print("=== GEOMETRIC PROPERTIES ===")
S1 = Slab("S1",5200,2250,125)
#S1 = Slab("S1",6800,6800,175)
lb = S1.L #larger dimension
la = S1.W #shorter dimension
print(f"1. L = {S1.L}mm, W = {S1.W}mm, thk = {S1.thk}mm.")
beta = S1.L / S1.W #Ratio of longer to shorter slab dim
print(f"2. L2/L1 = {rd(beta)}, ratio of longer to shorter slab clear span.")

#Beam-to-slab Stiffness Ratio, alpha_f
af_n = B_nort.alpha_f(S1.Ics_W_whole())
af_s = B_sout.alpha_f(S1.Ics_W_whole())
af_e = B_east.alpha_f(S1.Ics_L_whole())
af_w = B_west.alpha_f(S1.Ics_L_half())

alpha_fm = rd((af_n+af_s+af_e+af_w)/4)
print(f"3. alpha_fm = {alpha_fm}, ave. rel.stiff. of beam to slab")


=== MATERIAL PROPERTIES ===
fc = 27MPa | fy = 230MPa | Ec = 24.42GPa
=== GEOMETRIC PROPERTIES ===
1. L = 5200mm, W = 2250mm, thk = 125mm.
2. L2/L1 = 2.31, ratio of longer to shorter slab clear span.
B_north, alpha_f = 43.74
B_south, alpha_f = 43.74
B_east, alpha_f = 18.93
B_west, alpha_f = 28.39
3. alpha_fm = 33.7, ave. rel.stiff. of beam to slab


In [6]:
#Slab Minimum Thickness per 8.3.1.2 ACI318M-14
print("=== MIN. THICKNESS (8.3.1.2) ===")
if alpha_fm > 0.20 and alpha_fm < 2:
    print(f"1. case 1 governs: 0.20 < alpha_fm < 2.0")
    h_min = rd(max( S1.L * (0.8 + fy/1400) / (36+5*(S1.L/S1.W)*(alpha_fm - 0.2)) , 125)) #mm
else: 
    print(f"1. case 2 governs: alpha_fm > 2.0")
    h_min = rd(max( S1.L * (0.8 + fy/1400) / (36+9*(S1.L/S1.W)) , 90)) #mm
    
if S1.thk > h_min:
    print(f"2. Minimum Slab Thickness (ACI318M-14, 8.3.1.2) = {h_min}mm ({mm_to_in(h_min)}in). OKAY!")
else:
    exit("NON-COMPLIANCE WITH MINIMUM THICKNESS.")
#Factored Uniform Load
print("=== APPLIED LOADS ===")
DL = rd(24*max(h_min,S1.thk)*.001+2.2) #selfweight of slab, kPa
LL = 2.4 #kPa
qa = DL + LL
qu = 1.2*DL + 1.6*LL
print(f"1. DL = {DL}kPa ({kPa_to_psf(DL)}psf) | LL = {LL}kPa ({kPa_to_psf(LL)}psf)")
print(f"2. Service Uniform Pressure, qa = {rd(qa)}kPa ({kPa_to_psf(qa)}psf)")
print(f"3. Factored Uniform Pressure, qu = {rd(qu)}kPa ({kPa_to_psf(qu)}psf)") #kPa

=== MIN. THICKNESS (8.3.1.2) ===
1. case 2 governs: alpha_fm > 2.0
2. Minimum Slab Thickness (ACI318M-14, 8.3.1.2) = 90mm (3.54in). OKAY!
=== APPLIED LOADS ===
1. DL = 5.2kPa (108.58psf) | LL = 2.4kPa (50.11psf)
2. Service Uniform Pressure, qa = 7.6kPa (158.69psf)
3. Factored Uniform Pressure, qu = 10.08kPa (210.47psf)


In [7]:
#Minimum reinforcement near tension face for each dir
print("=== MIN. REINF. NEAR TENSION FACE, EA.DIR. (8.6.1) ===")
if fy < 420:
    Asmin = rd(0.0020 * 1000 * S1.thk)
    print(f"1. Asmin = 0.0020Ag = {Asmin}mm2 per mtr width of slab")
else:
    Asmin = rd( max( 0.0018*420*1000*S1.thk/fy , 0.0014*1000*S1.thk ) )
    print(f"1. Asmin = {Asmin}mm2 per mtr width of slab")

#Minimum spacing of flexural reinforcement at crit sections
print("1. Max. spacing of flex. reinf. at crit. sec. must not exceed 2*slab_thk per 8.7.3")
s_max = int(2*S1.thk) #mm
print(f"2. s_max = {s_max}mm")

#Corner restraint in slab
print("=== CORNER RESTRAINT IN SLAB (8.7.3) ===")
lcor = int(rd(lb / 5)) #mm
print(f"1. Length corner bar, lcor = {lcor}mm")
Mu_cor = rd((1/2)*qu*(lcor*0.001)**2) #kN-m
print(f"2. Mu_cor = {Mu_cor}kN-m")


=== MIN. REINF. NEAR TENSION FACE, EA.DIR. (8.6.1) ===
1. Asmin = 0.0020Ag = 250.0mm2 per mtr width of slab
1. Max. spacing of flex. reinf. at crit. sec. must not exceed 2*slab_thk per 8.7.3
2. s_max = 250mm
=== CORNER RESTRAINT IN SLAB (8.7.3) ===
1. Length corner bar, lcor = 1040mm
2. Mu_cor = 5.45kN-m


In [8]:
#Case: Simply Supported Square Slab, if only bending moments were present
print("=== CASE: SIMPLY SUPPORTED SQUARE SLAB ===")
print("1. Twisting moment relieves 25 percent of bending moment.")
print("2. Bending moment formula: (qu/2)*l**2 / 8 less 25%")
Mu_square = ( ((qu*0.5)*(lb*0.001)**2) / 8 ) * 0.75 #kN-m
print(f"3. Mu_squareslab = {rd(Mu_square)}kN-m per mtr width")

=== CASE: SIMPLY SUPPORTED SQUARE SLAB ===
1. Twisting moment relieves 25 percent of bending moment.
2. Bending moment formula: (qu/2)*l**2 / 8 less 25%
3. Mu_squareslab = 12.78kN-m per mtr width


In [9]:
#Total Statical Moment, Mo (CONSIDER BENDING OF THE LONGER SPAN)
print("=== TOTAL STATICAL MOMENT, Mo ===")
Mo_LDir = qu*(S1.W+(B_nort.dimy+B_sout.dimy)*.5)*.001*(S1.L*.001)**2 / 8 #kN-m
print(f"1a. Total Statical Moment, Mo_LDir = {rd(Mo_LDir)}kN-m ({kNm_to_kipft(Mo_LDir)}kip-ft) long dir.")
Mo_SDir = qu*(S1.L+(B_east.dimy+B_west.dimy)*.5)*.001*(S1.W*.001)**2 / 8 #kN-m
print(f"1b. Total Statical Moment, Mo_SDir = {rd(Mo_SDir)}kN-m ({kNm_to_kipft(Mo_SDir)}kip-ft) shrt dir.")
#Widths of Middle and Column Strip...
width_cs = rd(min(0.5*S1.L,0.5*S1.W)) # 8.4.1.5
print(f"2a. Width of Column Strip: {width_cs}mm ({mm_to_in(width_cs/12)}ft)") 
width_ms_sdir = rd(S1.L - width_cs) #mm
width_ms_ldir = rd(S1.W - width_cs) #mm
print(f"2b. Width of Middle Strip, for sdir: {width_ms_sdir}mm ({mm_to_in(width_ms_sdir/12)}ft)")
print(f"2c. Width of Middle Strip, for ldir: {width_ms_ldir}mm ({mm_to_in(width_ms_ldir/12)}ft)")

=== TOTAL STATICAL MOMENT, Mo ===
1a. Total Statical Moment, Mo_LDir = 88.58kN-m (65.34kip-ft) long dir.
1b. Total Statical Moment, Mo_SDir = 35.4kN-m (26.11kip-ft) shrt dir.
2a. Width of Column Strip: 1125.0mm (3.69ft)
2b. Width of Middle Strip, for sdir: 4075.0mm (13.37ft)
2c. Width of Middle Strip, for ldir: 1125.0mm (3.69ft)


In [10]:
import k_cs
k_eneg_cs = k_cs.k_eneg_cs
k_poss_cs = k_cs.k_poss_cs
k_ineg_cs = k_cs.k_ineg_cs

#Parameters for Exterior Panel, Calc of CS Ext, Pos, and Int % of Mo

betat = rd(B_east.beta_t(S1.thk,S1.Ics_W_whole()))
l2ol1 = rd(S1.W/S1.L)
a_l2ol1 = rd(af_w*l2ol1)
print("=== ke for Ext. Panel ===")
print(f"betat = {betat} | a_l2ol1 = {a_l2ol1} | af_w = {af_w} | l2ol1 = {l2ol1}")

#Exterior Panel Exterior Negative Moment for CS
ke_e = k_eneg_cs(betat,a_l2ol1,l2ol1)

#Exterior Panel Positive Moment for CS
ke_p = k_poss_cs(a_l2ol1,l2ol1)

#Exterior Panel Interior Negative Moment for CS
ke_i = k_ineg_cs(a_l2ol1,l2ol1)
print(f"ke_e = {ke_e} | ke_p = {ke_p} | ke_i = {ke_i}")

#Interior Panel Negative Moment for CS
kn = k_ineg_cs(af_e*S1.L/S1.W,S1.L/S1.W)

#Interior Panel Positive Moment for CS
kp = k_poss_cs(af_e*S1.L/S1.W,S1.L/S1.W)
print("=== k for Int. Panel ===")
print(f"af_e = {af_e} | l2ol1 = {rd(S1.L/S1.W)} | a_l2ol1 = {rd(af_e*S1.L/S1.W)}")
print(f"kn = {kn} | kp = {kp}")


=== ke for Ext. Panel ===
betat = 8.76 | a_l2ol1 = 12.21 | af_w = 28.39 | l2ol1 = 0.43
ke_e = 0.9 | ke_p = 0.9 | ke_i = 0.9
=== k for Int. Panel ===
af_e = 18.93 | l2ol1 = 2.31 | a_l2ol1 = 43.75
kn = 0.45 | kp = 0.45


In [11]:
#Interior Panel Moments Distribution of Mo
print("=== LDIR: Distribuition of Mo for Interior Panels (8.10.4.1) ===")
Mol_neg = rd(0.65*Mo_LDir) #at supports/ends of slab
Mol_pos = rd(0.35*Mo_LDir) #at midspan of slab
print(f" 1a. Interior Panel Moment (negative), Mol_neg = {Mol_neg}kNm ({kNm_to_kipft(Mol_neg)}kipft)")
print(f" 1b. Interior Panel Moment (positive), Mol_pos = {Mol_pos}kNm ({kNm_to_kipft(Mol_pos)}kipft)")
#Neg Mu, Column Strip, Table 8.10.5.1
Mol_neg_cs = rd(kn*Mol_neg)
Mol_neg_cs_beam = rd(0.85*Mol_neg_cs)
Mol_neg_cs_slab = rd(0.15*Mol_neg_cs)
#Pos Mu, Column Strip, Table 8.10.5.5
Mol_pos_cs = rd(kp*Mol_pos)
Mol_pos_cs_beam = rd(0.85*Mol_pos_cs)
Mol_pos_cs_slab = rd(0.15*Mol_pos_cs)
#Interior Panel, Middle Strip
Mol_neg_ms = rd(Mol_neg - Mol_neg_cs) #8.10.6
#Interior Panel, Middle Strip
Mol_pos_ms = rd(Mol_pos - Mol_pos_cs) #8.10.6

print("=== SDIR: Distribuition of Mo for Interior Panels (8.10.4.1) ===")
Mos_neg = rd(0.65*Mo_SDir) #at supports/ends of slab
Mos_pos = rd(0.35*Mo_SDir) #at midspan of slab
print(f" 1a. Interior Panel Moment (negative), Mos_neg = {Mos_neg}kNm ({kNm_to_kipft(Mos_neg)}kipft)")
print(f" 1b. Interior Panel Moment (positive), Mos_pos = {Mos_pos}kNm ({kNm_to_kipft(Mos_pos)}kipft)")
#Neg Mu, Column Strip, Table 8.10.5.1
Mos_neg_cs = rd(kn*Mos_neg)
Mos_neg_cs_beam = rd(0.85*Mos_neg_cs)
Mos_neg_cs_slab = rd(0.15*Mos_neg_cs)
#Pos Mu, Column Strip, Table 8.10.5.5
Mos_pos_cs = rd(kp*Mos_pos)
Mos_pos_cs_beam = rd(0.85*Mos_pos_cs)
Mos_pos_cs_slab = rd(0.15*Mos_pos_cs)
#Interior Panel, Middle Strip
Mos_neg_ms = rd(Mos_neg - Mos_neg_cs) #8.10.6
Mos_pos_ms = rd(Mos_pos - Mos_pos_cs) #8.10.6
def print_intpanel_stripdist():
    print("LDIR")
    print(f"  2a. IPM_neg, Column Strip, Mol_neg_cs = {Mol_neg_cs}kNm ({kNm_to_kipft(Mol_neg_cs)}kipft)")
    print(f"    2a.1. Mol_neg_cs_beam (85%, 8.10.5.7): {Mol_neg_cs_beam}kNm ({kNm_to_kipft(Mol_neg_cs_beam)}kipft)")
    print(f"    2a.1. Mol_neg_cs_slab (15%, 8.10.5.7): {Mol_neg_cs_slab}kNm ({kNm_to_kipft(Mol_neg_cs_slab)}kipft)")
    print(f"  2b. IPM_neg, Middle Strip, Mol_neg_ms = {Mol_neg_ms}kNm ({kNm_to_kipft(Mol_neg_ms)}kipft)")
    print(f"  2c. IPM_pos, Column Strip, Mol_pos_cs = {Mol_pos_cs}kNm ({kNm_to_kipft(Mol_pos_cs)}kipft)")
    print(f"    2c.1. Mol_pos_cs_beam (85%, 8.10.5.7): {Mol_pos_cs_beam}kNm ({kNm_to_kipft(Mol_pos_cs_beam)}kipft)")
    print(f"    2c.1. Mol_pos_cs_slab (15%, 8.10.5.7): {Mol_pos_cs_slab}kNm ({kNm_to_kipft(Mol_pos_cs_slab)}kipft)")
    print(f"  2d. IPM_pos, Mid Strip, Mol_pos_ms = {Mol_pos_ms}kNm ({kNm_to_kipft(Mol_pos_ms)}kipft)")
    print("SDIR")
    print(f"  2a. IPM_neg, Column Strip, Mos_neg_cs = {Mos_neg_cs}kNm ({kNm_to_kipft(Mos_neg_cs)}kipft)")
    print(f"    2a.1. Mos_neg_cs_beam (85%, 8.10.5.7): {Mos_neg_cs_beam}kNm ({kNm_to_kipft(Mos_neg_cs_beam)}kipft)")
    print(f"    2a.1. Mos_neg_cs_slab (15%, 8.10.5.7): {Mos_neg_cs_slab}kNm ({kNm_to_kipft(Mos_neg_cs_slab)}kipft)")
    print(f"  2b. IPM_neg, Middle Strip, Mos_neg_ms = {Mos_neg_ms}kNm ({kNm_to_kipft(Mos_neg_ms)}kipft)")
    print(f"  2c. IPM_pos, Column Strip, Mos_pos_cs = {Mos_pos_cs}kNm ({kNm_to_kipft(Mos_pos_cs)}kipft)")
    print(f"    2c.1. Mos_pos_cs_beam (85%, 8.10.5.7): {Mos_pos_cs_beam}kNm ({kNm_to_kipft(Mos_pos_cs_beam)}kipft)")
    print(f"    2c.1. Mos_pos_cs_slab (15%, 8.10.5.7): {Mos_pos_cs_slab}kNm ({kNm_to_kipft(Mos_pos_cs_slab)}kipft)")
    print(f"  2d. IPM_pos, Mid Strip, Mos_pos_ms = {Mos_pos_ms}kNm ({kNm_to_kipft(Mos_pos_ms)}kipft)")


=== LDIR: Distribuition of Mo for Interior Panels (8.10.4.1) ===
 1a. Interior Panel Moment (negative), Mol_neg = 57.58kNm (42.47kipft)
 1b. Interior Panel Moment (positive), Mol_pos = 31.0kNm (22.86kipft)
=== SDIR: Distribuition of Mo for Interior Panels (8.10.4.1) ===
 1a. Interior Panel Moment (negative), Mos_neg = 23.01kNm (16.97kipft)
 1b. Interior Panel Moment (positive), Mos_pos = 12.39kNm (9.14kipft)


In [12]:
print("* * * * * * LDIR - INT PANEL * * * * * * *")
Mol_summary_ip = {
    "CS Beam Moment"   :pd.Series([Mol_pos_cs_beam,Mol_neg_cs_beam],index=["pos","neg"]),
    "CS Slab Moment":pd.Series([Mol_pos_cs_slab, Mol_neg_cs_slab],index=["pos","neg"]),
    "MS Slab Moment":pd.Series([Mol_pos_ms,Mol_neg_ms],index=["pos","neg"])
}
df_Mol_summary_ip = pd.DataFrame(Mol_summary_ip)
print(df_Mol_summary_ip)
print("* * * * * * SDIR - INT PANEL * * * * * * *")
Mos_summary_ip = {
    "CS Beam Moment"   :pd.Series([Mos_pos_cs_beam, Mos_neg_cs_beam],index=["pos","neg"]),
    "CS Slab Moment":pd.Series(   [Mos_pos_cs_slab, Mos_neg_cs_slab],index=["pos","neg"]),
    "MS Slab Moment":pd.Series(   [Mos_pos_ms,      Mos_neg_ms],index=["pos","neg"])
}
df_Mos_summary_ip = pd.DataFrame(Mos_summary_ip)
print(df_Mos_summary_ip)

* * * * * * LDIR - INT PANEL * * * * * * *
     CS Beam Moment  CS Slab Moment  MS Slab Moment
pos           11.86            2.09           17.05
neg           22.02            3.89           31.67
* * * * * * SDIR - INT PANEL * * * * * * *
     CS Beam Moment  CS Slab Moment  MS Slab Moment
pos            4.74            0.84            6.81
neg            8.80            1.55           12.66


In [13]:
#End span Moments Distribution of Mo ACI318M-14 Table 8.10.4.2
print("=== LDIR: Distribution of Mo for End Span with beams between all supports ===")
Mol_eneg = rd(0.16*Mo_LDir)
Mol_ineg = rd(0.70*Mo_LDir)
Mol_poss = rd(0.57*Mo_LDir)
print(f" End Span Panel Moment (negative exterior), Mo_eneg = {Mol_eneg}kNm.")
print(f" End Span Panel Moment (negative interior), Mo_ineg = {Mol_ineg}kNm.")
print(f" End Span Panel Moment (positive), Mo_poss = {Mol_poss}kNm.")

Mol_eneg_cs = rd(ke_e*Mol_eneg) #T8.10.5.2
Mol_eneg_cs_beam = rd(0.85*Mol_eneg_cs)
Mol_eneg_cs_slab = rd(0.15*Mol_eneg_cs)

Mol_ineg_cs = rd(ke_i*Mol_ineg) #T8.10.5.1
Mol_ineg_cs_beam = rd(0.85*Mol_ineg_cs)
Mol_ineg_cs_slab = rd(0.15*Mol_ineg_cs)

Mol_poss_cs = rd(ke_p*Mol_poss)
Mol_poss_cs_beam = rd(0.85*Mol_poss_cs)
Mol_poss_cs_slab = rd(0.15*Mol_poss_cs)

Mol_eneg_ms = rd(Mol_eneg - Mol_eneg_cs)
Mol_ineg_ms = rd(Mol_ineg - Mol_ineg_cs)
Mol_poss_ms = rd(Mol_poss - Mol_poss_cs)

#SHORT DIRECTION 
print("=== SDIR: Distribution of Mo for End Span with beams between all supports ===")
Mos_eneg = rd(0.16*Mo_SDir)
Mos_ineg = rd(0.70*Mo_SDir)
Mos_poss = rd(0.57*Mo_SDir)
print(f" End Span Panel Moment (negative exterior), Mos_eneg = {Mos_eneg}kNm.")
print(f" End Span Panel Moment (negative interior), Mos_ineg = {Mos_ineg}kNm.")
print(f" End Span Panel Moment (positive), Mos_epos = {Mos_poss}kNm.")

Mos_eneg_cs = rd(ke_e*Mos_eneg)
Mos_eneg_cs_beam = rd(0.85*Mos_eneg_cs)
Mos_eneg_cs_slab = rd(0.15*Mos_eneg_cs)

Mos_ineg_cs = rd(ke_i*Mos_ineg)
Mos_ineg_cs_beam = rd(0.85*Mos_ineg_cs)
Mos_ineg_cs_slab = rd(0.15*Mos_ineg_cs)

Mos_poss_cs = rd(ke_p*Mos_poss)
Mos_poss_cs_beam = rd(0.85*Mos_poss_cs)
Mos_poss_cs_slab = rd(0.15*Mos_poss_cs)

Mos_eneg_ms = rd(Mos_eneg - Mos_eneg_cs)
Mos_ineg_ms = rd(Mos_ineg - Mos_ineg_cs)
Mos_poss_ms = rd(Mos_poss - Mos_poss_cs)
def print_extpanel_stripdist():
    print("LDIR")
    print(f"    1. ESPM_eneg, Col Strip, Mol_eneg_cs = {Mol_eneg_cs}kNm ({kNm_to_kipft(Mol_eneg_cs)}kipft)")
    print(f"      1a. Mol_eneg_cs_beam (85%, 8.10.5.7): {Mol_eneg_cs_beam}kNm ({kNm_to_kipft(Mol_eneg_cs_beam)}kipft)")
    print(f"      1b. Mol_eneg_cs_slab (15%, 8.10.5.7): {Mol_eneg_cs_slab}kNm ({kNm_to_kipft(Mol_eneg_cs_slab)}kipft)")
    print(f"    2. ESPM_ineg, Col Strip, Mol_ineg_cs = {Mol_ineg_cs}kNm ({kNm_to_kipft(Mol_ineg_cs)}kipft)")
    print(f"      2a. Mol_ineg_cs_beam (85%, 8.10.5.7): {Mol_ineg_cs_beam}kNm ({kNm_to_kipft(Mol_ineg_cs_beam)}kipft)")
    print(f"      2b. Mol_ineg_cs_slab (15%, 8.10.5.7): {Mol_ineg_cs_slab}kNm ({kNm_to_kipft(Mol_ineg_cs_slab)}kipft)")
    print(f"    3. ESPM_pos, Col Strip, Mol_poss_cs = {Mol_poss_cs}kNm ({kNm_to_kipft(Mol_poss_cs)}kipft)")
    print(f"      3a. Mol_poss_cs_beam (85%, 8.10.5.7): {Mol_poss_cs_beam}kNm ({kNm_to_kipft(Mol_poss_cs_beam)}kipft)")
    print(f"      3b. Mol_poss_cs_slab (15%, 8.10.5.7): {Mol_poss_cs_slab}kNm ({kNm_to_kipft(Mol_poss_cs_slab)}kipft)")
    print(f"    1. ESPM_eneg, Mid Strip, Mol_eneg_ms = {Mol_eneg_ms}kNm ({kNm_to_kipft(Mol_eneg_ms)}kipft)")
    print(f"    2. ESPM_ineg, Mid Strip, Mol_ineg_ms = {Mol_ineg_ms}kNm ({kNm_to_kipft(Mol_ineg_ms)}kipft)")
    print(f"    3. ESPM_pos, Mid Strip, Mol_poss_ms = {Mol_poss_ms}kNm ({kNm_to_kipft(Mol_poss_ms)}kipft)")
    print("SDIR")
    print(f"    1. ESPM_eneg, Col Strip, Mos_eneg_cs = {Mos_eneg_cs}kNm ({kNm_to_kipft(Mos_eneg_cs)}kipft)")
    print(f"      1a. Mos_eneg_cs_beam (85%, 8.10.5.7): {Mos_eneg_cs_beam}kNm ({kNm_to_kipft(Mos_eneg_cs_beam)}kipft)")
    print(f"      1b. Mos_eneg_cs_slab (15%, 8.10.5.7): {Mos_eneg_cs_slab}kNm ({kNm_to_kipft(Mos_eneg_cs_slab)}kipft)")
    print(f"    2. ESPM_ineg, Col Strip, Mos_ineg_cs = {Mos_ineg_cs}kNm ({kNm_to_kipft(Mos_ineg_cs)}kipft)")
    print(f"      2a. Mos_ineg_cs_beam (85%, 8.10.5.7): {Mos_ineg_cs_beam}kNm ({kNm_to_kipft(Mos_ineg_cs_beam)}kipft)")
    print(f"      2b. Mos_ineg_cs_slab (15%, 8.10.5.7): {Mos_ineg_cs_slab}kNm ({kNm_to_kipft(Mos_ineg_cs_slab)}kipft)")
    print(f"    3. ESPM_pos, Col Strip, Mos_poss_cs = {Mos_poss_cs}kNm ({kNm_to_kipft(Mos_poss_cs)}kipft)")
    print(f"      3a. Mos_poss_cs_beam (85%, 8.10.5.7): {Mos_poss_cs_beam}kNm ({kNm_to_kipft(Mos_poss_cs_beam)}kipft)")
    print(f"      3b. Mos_poss_cs_slab (15%, 8.10.5.7): {Mos_poss_cs_slab}kNm ({kNm_to_kipft(Mos_poss_cs_slab)}kipft)")
    print(f"    1. ESPM_eneg, Mid Strip, Mos_eneg_ms = {Mos_eneg_ms}kNm ({kNm_to_kipft(Mos_eneg_ms)}kipft)")
    print(f"    2. ESPM_ineg, Mid Strip, Mos_ineg_ms = {Mos_ineg_ms}kNm ({kNm_to_kipft(Mos_ineg_ms)}kipft)")
    print(f"    3. ESPM_pos, Mid Strip, Mos_poss_ms = {Mos_poss_ms}kNm ({kNm_to_kipft(Mos_poss_ms)}kipft)")

=== LDIR: Distribution of Mo for End Span with beams between all supports ===
 End Span Panel Moment (negative exterior), Mo_eneg = 14.17kNm.
 End Span Panel Moment (negative interior), Mo_ineg = 62.01kNm.
 End Span Panel Moment (positive), Mo_poss = 50.49kNm.
=== SDIR: Distribution of Mo for End Span with beams between all supports ===
 End Span Panel Moment (negative exterior), Mos_eneg = 5.66kNm.
 End Span Panel Moment (negative interior), Mos_ineg = 24.78kNm.
 End Span Panel Moment (positive), Mos_epos = 20.18kNm.


In [14]:
print("* * * * * * LDIR - EXT PANEL * * * * * * *")
Mol_summary = {
    "CS Beam Moment":pd.Series([Mol_ineg_cs_beam,Mol_eneg_cs_beam,Mol_poss_cs_beam],index=["int_neg","ext_neg","pos"]),
    "CS Slab Moment":pd.Series([Mol_ineg_cs_slab,Mol_eneg_cs_slab,Mol_poss_cs_slab],index=["int_neg","ext_neg","pos"]),
    "MS Slab Moment":pd.Series([Mol_ineg_ms,     Mol_eneg_ms,     Mol_poss_ms],     index=["int_neg","ext_neg","pos"])
}
df_Mol_summary = pd.DataFrame(Mol_summary)
print(df_Mol_summary)
print("* * * * * * SDIR - EXT PANEL * * * * * * *")
Mos_summary = {
    "CS Beam Moment":pd.Series([Mos_ineg_cs_beam,Mos_eneg_cs_beam,Mos_poss_cs_beam],index=["int_neg","ext_neg","pos"]),
    "CS Slab Moment":pd.Series([Mos_ineg_cs_slab,Mos_eneg_cs_slab,Mos_poss_cs_slab],index=["int_neg","ext_neg","pos"]),
    "MS Slab Moment":pd.Series([Mos_ineg_ms     ,Mos_eneg_ms,     Mos_poss_ms],     index=["int_neg","ext_neg","pos"])
}
df_Mos_summary = pd.DataFrame(Mos_summary)
print(df_Mos_summary)

* * * * * * LDIR - EXT PANEL * * * * * * *
         CS Beam Moment  CS Slab Moment  MS Slab Moment
int_neg           47.44            8.37            6.20
ext_neg           10.84            1.91            1.42
pos               38.62            6.82            5.05
* * * * * * SDIR - EXT PANEL * * * * * * *
         CS Beam Moment  CS Slab Moment  MS Slab Moment
int_neg           18.96            3.35            2.48
ext_neg            4.33            0.76            0.57
pos               15.44            2.72            2.02


In [15]:
#Strip / width
bw_cs = width_cs #mm
bw_ms_sdir = width_ms_sdir #mm
bw_ms_ldir = width_ms_ldir #mm
print(f"bw_ms_sdir = {bw_ms_sdir}mm ({mm_to_in(bw_ms_sdir)}in) | bw_ms_ldir = {bw_ms_ldir}mm ({mm_to_in(bw_ms_ldir)}in) | bw_cs = {bw_cs}mm ({mm_to_in(bw_cs)}in)")

#LONG DIR: RSB SPACING
#Moment for bottom reinf
LDir_ms_Mo_bot = max(Mol_poss_ms,      Mol_pos_ms)
LDir_cs_Mo_bot = max(Mol_poss_cs_slab, Mol_pos_cs_slab)
#moment for top reinf
LDir_ms_Mo_top = max(Mol_ineg_ms,      Mol_eneg_ms,      Mol_neg_ms)
LDir_cs_Mo_top = max(Mol_ineg_cs_slab, Mol_eneg_cs_slab, Mol_neg_cs_slab)

#SHRT DIR: RSB SPACING
#Moment for bottom reinf
SDir_ms_Mo_bot = max(Mos_poss_ms,      Mos_pos_ms)
SDir_cs_Mo_bot = max(Mos_poss_cs_slab, Mos_pos_cs_slab)
#Moment for top reinf
SDir_ms_Mo_top = max(Mos_ineg_ms,      Mos_eneg_ms,      Mos_neg_ms)
SDir_cs_Mo_top = max(Mos_ineg_cs_slab, Mos_eneg_cs_slab, Mos_neg_cs_slab)

print(f"LDir: ms_bot = {LDir_ms_Mo_bot}kNm | ms_top = {LDir_ms_Mo_top}kN | cs_bot = {LDir_cs_Mo_bot}kNm | cs_top = {LDir_cs_Mo_top}kNm")
print(f"SDir: ms_bot = {SDir_ms_Mo_bot}kNm | ms_top = {SDir_ms_Mo_top}kN |cs_bot = {SDir_cs_Mo_bot}kNm | cs_top = {SDir_cs_Mo_top}kNm")


bw_ms_sdir = 4075.0mm (160.43in) | bw_ms_ldir = 1125.0mm (44.29in) | bw_cs = 1125.0mm (44.29in)
LDir: ms_bot = 17.05kNm | ms_top = 31.67kN | cs_bot = 6.82kNm | cs_top = 8.37kNm
SDir: ms_bot = 6.81kNm | ms_top = 12.66kN |cs_bot = 2.72kNm | cs_top = 3.35kNm


In [16]:
#Tensile Reinforcement Detail
d_main = 12     #diameter of main bars, mm
sc = 25         #steel cover

class StopExecution(Exception):
    def _render_traceback_(self):
        return []
    
def exit():
    raise StopExecution

def MomentCap(bw,h,layer, n1, n2, Mu):
    if layer > 1:
        As1 = n1*pi*0.25*d_main**2
        As2 = n2*pi*0.25*d_main**2
        As = rd(As1 + As2)
        y = (As1*0 + As2*(d_main + 25))/As
        dt = h - sc - d_main/2      #reinforcement farthest from the compression face
        d = rd(dt - y)                          #centroid of steel
        print(f"1. dmain = {d_main} | n1 = {n1}pcs, n2 = {n2}pcs | d = {d}mm")
    else:
        dt = h - sc - d_main/2      #reinforcement farthest from the compression face
        d = dt                              #centroid of steel
        As = rd( pi*d_main**2*0.25*n1 )
        print(f"1. dmain = {d_main} | n = {n1}pcs | dt = {dt}mm | bw = {bw}mm")
    
    s2 = round((bw)/(n1))        #spacing of the long reinf., mm
    print(f"2. As = {As}mm2 | spacing (mainbar) = {s2}mm.")
    #Beta Factor
    if fc <= 28:
        beta1 = 0.85
    elif 28 < fc < 55:
        beta1 = round(0.85 - 0.05*(fc-28)/7,4)
    else:
        beta1 = 0.65
    #Equivalent compression block
    a = As*fy/(0.85*fc*bw)
    c = a/beta1
    #Rho for Balanced Strain Condition
    eu = 0.003      #crushing strain of concrete
    ety = 0.002      #ACI 21.2.2.1, net tensile strain value in the extreme layer of long ten reinf. if compression controlled
    ey = fy/Es     
    rho_b = round(0.85*beta1*fc*eu/(fy*(eu+ety)),5)
    #Rho for Actual Strain Condition
    et = eu*(dt-c)/c    #net tensile strain in extreme layer of long ten reinf.
    rho_act = round(0.85*beta1*fc*dt*eu/(fy*d*(eu+et)),5)
    #Rho for maximum reinforcement ratio for a tension-controlled beam
    rho_ten = round(0.85*beta1*fc*eu/(fy*(eu+0.005)),5)
    print(f"3. rho_bal: {rho_b} | rho_act: {rho_act} | rho_ten: {rho_ten}")
    def phi():
        if et <= ety:
            phi = 0.65
        elif ety < et < 0.005:
            phi = 0.65 + 0.25*(et - ey)/(0.005 - ey)
        else:
            phi = 0.90
        return phi
    print(f"4. phi_flex = {phi()} | beta1 = {beta1}")
    #Bending Capacity
    Mn = rd(0.85*fc*a*bw*(d-a/2)*(0.001**2))      #Bending Capacity, kN-m
    phiMn = rd(phi()*Mn) #kN-m
    if phiMn > Mu:
        print(f"5. phiMn = {phiMn}kNm ({kNm_to_kipft(phiMn)}kipft) > Mu = {Mu}kNm ({kNm_to_kipft(Mu)}kipft). SAFE!")
        #Check for rho_min
        if 0.002 < rho_act: #rho_min = 0.002
            pass
        else:
            print("NON COMPLIANT WITH RHO_MIN (0.002)")
            exit()
        #Check for max spacing
        if s2 < int(min(5*h,450,s_max)): #max spacing per  24.4.3.2-3
            pass
        else:
            print("NON COMPLIANT WITH MAX SPACING (5*h, 450)")
            exit()
    else:
        print(f"5. phiMn = {phiMn}kNm ({kNm_to_kipft(phiMn)}kipft) < Mu = {Mu}kNm ({kNm_to_kipft(Mu)}kipft). UNSAFE!")
        exit()
    return Mn, phiMn, d, s2, n1, n2, layer, rho_act

In [25]:
#MomentCapacity
print("=== LONG DIR: MS BOT BARS ===")
lms_bot = MomentCap(bw_ms_ldir,S1.thk-d_main,1,10,0,LDir_ms_Mo_bot)
print("=== LONG DIR: MS TOP BARS ===")
lms_top = MomentCap(bw_ms_ldir,S1.thk-d_main,1,19,0,LDir_ms_Mo_top)
print("=== LONG DIR: CS BOT BARS ===")
lcs_bot = MomentCap(bw_cs,S1.thk-d_main,1,5,0,LDir_cs_Mo_bot)
print("=== LONG DIR: CS TOP BARS===")
lcs_top = MomentCap(bw_cs,S1.thk-d_main,1,5,0,LDir_cs_Mo_top)

=== LONG DIR: MS BOT BARS ===
1. dmain = 12 | n = 10pcs | dt = 82.0mm | bw = 1125.0mm
2. As = 1130.97mm2 | spacing (mainbar) = 112mm.
3. rho_bal: 0.05089 | rho_act: 0.01226 | rho_ten: 0.03181
4. phi_flex = 0.9 | beta1 = 0.85
5. phiMn = 18.02kNm (13.29kipft) > Mu = 17.05kNm (12.58kipft). SAFE!
=== LONG DIR: MS TOP BARS ===
1. dmain = 12 | n = 19pcs | dt = 82.0mm | bw = 1125.0mm
2. As = 2148.85mm2 | spacing (mainbar) = 59mm.
3. rho_bal: 0.05089 | rho_act: 0.02329 | rho_ten: 0.03181
4. phi_flex = 0.9 | beta1 = 0.85
5. phiMn = 32.22kNm (23.76kipft) > Mu = 31.67kNm (23.36kipft). SAFE!
=== LONG DIR: CS BOT BARS ===
1. dmain = 12 | n = 5pcs | dt = 82.0mm | bw = 1125.0mm
2. As = 565.49mm2 | spacing (mainbar) = 225mm.
3. rho_bal: 0.05089 | rho_act: 0.00613 | rho_ten: 0.03181
4. phi_flex = 0.9 | beta1 = 0.85
5. phiMn = 9.31kNm (6.87kipft) > Mu = 6.82kNm (5.03kipft). SAFE!
=== LONG DIR: CS TOP BARS===
1. dmain = 12 | n = 5pcs | dt = 82.0mm | bw = 1125.0mm
2. As = 565.49mm2 | spacing (mainbar) = 2

In [34]:
print("=== SHRT DIR: MS BOT BARS ===")
sms_bot = MomentCap(bw_ms_sdir,S1.thk,1,17,0,SDir_ms_Mo_bot)
print("=== SHRT DIR: MS TOP BARS ===")
sms_top = MomentCap(bw_ms_sdir,S1.thk,1,17,0,SDir_ms_Mo_top)
print("=== SHRT DIR: CS BOT BARS ===")
scs_bot = MomentCap(bw_cs,S1.thk,1,5,0,SDir_cs_Mo_bot)
print("=== SHRT DIR: CS TOP BARS ===")
scs_top = MomentCap(bw_cs,S1.thk,1,5,0,SDir_cs_Mo_top)


=== SHRT DIR: MS BOT BARS ===
1. dmain = 12 | n = 17pcs | dt = 94.0mm | bw = 4075.0mm
2. As = 1922.65mm2 | spacing (mainbar) = 240mm.
3. rho_bal: 0.05089 | rho_act: 0.00502 | rho_ten: 0.03181
4. phi_flex = 0.9 | beta1 = 0.85
5. phiMn = 36.47kNm (26.9kipft) > Mu = 6.81kNm (5.02kipft). SAFE!
=== SHRT DIR: MS TOP BARS ===
1. dmain = 12 | n = 17pcs | dt = 94.0mm | bw = 4075.0mm
2. As = 1922.65mm2 | spacing (mainbar) = 240mm.
3. rho_bal: 0.05089 | rho_act: 0.00502 | rho_ten: 0.03181
4. phi_flex = 0.9 | beta1 = 0.85
5. phiMn = 36.47kNm (26.9kipft) > Mu = 12.66kNm (9.34kipft). SAFE!
=== SHRT DIR: CS BOT BARS ===
1. dmain = 12 | n = 5pcs | dt = 94.0mm | bw = 1125.0mm
2. As = 565.49mm2 | spacing (mainbar) = 225mm.
3. rho_bal: 0.05089 | rho_act: 0.00535 | rho_ten: 0.03181
4. phi_flex = 0.9 | beta1 = 0.85
5. phiMn = 10.71kNm (7.9kipft) > Mu = 2.72kNm (2.01kipft). SAFE!
=== SHRT DIR: CS TOP BARS ===
1. dmain = 12 | n = 5pcs | dt = 94.0mm | bw = 1125.0mm
2. As = 565.49mm2 | spacing (mainbar) = 225m

In [35]:
print("=== SUMMARY ===")
print(f"Design for SlabDimMax: {S1.L} x {S1.W}mm")
print(f"Governing Slab Thickness: {S1.thk}mm")
print(f"shrt_dir: {d_main}mm dia, topBars_spcd@{min(sms_top[3],scs_top[3])}mm, botBars_spcd@{min(sms_bot[3],scs_bot[3])}mm")
print(f"long_dir: {d_main}mm dia, topBars_spcd@{min(lms_top[3],lcs_top[3])}mm, botBars_spcd@{min(lms_bot[3],lcs_bot[3])}mm")

=== SUMMARY ===
Design for SlabDimMax: 5200 x 2250mm
Governing Slab Thickness: 125mm
shrt_dir: 12mm dia, topBars_spcd@225mm, botBars_spcd@225mm
long_dir: 12mm dia, topBars_spcd@59mm, botBars_spcd@112mm
